# 01 - Compute Long Term Averages
For each case, compute the global 3D (`nlat`, `nlon`, `z_t`) mean

## Imports
We include the lines at the beginning to make sure that any updates we make to the `analysis_config.yml` file are reflected in real time for this notebook

In [1]:
%load_ext autoreload
%autoreload 2

import intake
import ast
import yaml
from distributed import Client
from ncar_jobqueue import NCARCluster
import xarray as xr

## Spin up a Dask Cluster

In [2]:
cluster = NCARCluster()
cluster.scale(20)
client = Client(cluster)

/glade/u/home/mgrover/.local/lib/python3.9/site-packages/dask_jobqueue/core.py:19: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/glade/u/home/mgrover/.local/lib/python3.9/site-packages/dask_jobqueue/core.py:19: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/glade/u/home/mgrover/.local/lib/python3.9/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


In [3]:
client

<Client: 'tcp://10.12.206.42:46308' processes=0 threads=0, memory=0 B>

## Open the Intake-ESM Catalog
In the first notebook, we created a an `intake-esm` catalog which provides a means of accessing our data.

In [4]:
col = intake.open_esm_datastore(
    "/glade/work/mgrover/cesm-validation-catalog.json",
    csv_kwargs={"converters": {"variables": ast.literal_eval}},
    sep="/",
)
col

,unique
component,1
stream,4
date,2501
case,3
member_id,2
frequency,4
variables,545
path,11103


In [5]:
cat = col.search(
    stream='pop.h',
)
cat

,unique
component,1
stream,1
date,1200
case,3
member_id,2
frequency,1
variables,434
path,3600


### Subset the last 20 years of data

In [6]:
dates = sorted(cat.df.date.unique())

In [7]:
sub = cat.search(date=dates[-240:])

### Read in our dataset using `to_dataset_dict()`

In [11]:
dsets = sub.to_dataset_dict(cdf_kwargs={'use_cftime': True, 'chunks': {'time': 10}})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component/stream/case'


In [12]:
dsets.keys()

dict_keys(['ocn/pop.h/b1850.f19_g17.validation_mct.002', 'ocn/pop.h/b1850.f19_g17.validation_mct.004', 'ocn/pop.h/b1850.f19_g17.validation_nuopc.004'])

## Loop through the data and compute!
We are computing the average over time, and merging into a single dataset, subsetting for the variables specified in the `analysis_config.yml` file

In [77]:
with open("analysis_config.yml", mode="r") as fptr:
    analysis_config = yaml.safe_load(fptr)
variables = analysis_config['variables']
variables

['TEMP', 'SALT', 'FG_CO2', 'NH4', 'NO3', 'SiO3']

In [65]:
ds_list = []
for key in dsets.keys():
    ds = dsets[key]
    mean = ds.mean(dim='time')
    out = mean[variables].compute()
    for var in variables:
        out[var].attrs = mean[var].attrs
    out.attrs = ds.attrs
    out.attrs['intake_esm_varname'] = variables
    ds_list.append(out)

In [66]:
merged_ds = xr.concat(ds_list, dim='case')

### Make sure our variables keep their attributes

In [71]:
for var in variables:
    merged_ds[var].attrs = ds[var].attrs

We also want to make sure that we keep the title, or case information

In [73]:
cases = []
for ds in ds_list:
    cases.append(ds.title)

In [74]:
merged_ds['case'] = cases

### Export our data
We output our dataset to zarr!

In [76]:
merged_ds.to_zarr('cached_output/averages_year_081_100.zarr')